# Import

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# 1. データ読込

In [ ]:
# ワインデータセットをロード
wine = load_wine()
X = wine.data
y = wine.target

# データを訓練用とテスト用に分割 (層化抽出)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2, # 20%をテストデータに
    random_state=42, # 乱数シード固定
    stratify=y # ターゲット（ワインの３クラス）が均等になるようsplit
)

## 参考 データの意味
- `X` - 178行 × 13列
    - `alcohol` - アルコール度数
    - `malic_acid` - リンゴ酸含有量
    - `ash` - 灰分
    - `alcalinity_of_ash` - 灰分のアルカリ度
    - `magnesium` - マグネシウム含有量
    - `total_phenols` - 総フェノール含有量
    - `flavanoids` - フラバノイド含有量
    - `nonflavanoid_phenols` - 非フラバノイドフェノール含有量
    - `proanthocyanins` - プロアントシアニン含有量
    - `color_intensity` - 色の濃さ
    - `hue` - 色相
    - `od280/od315_of_diluted_wines` - 希釈ワインの280nm/315nmでの吸光度比
    - `proline` -プロリン含有量
- `y` - ワインの等級3種類のクラスラベル、これを当てます

In [ ]:
# PyTorchのテンソルに変換
# 特徴量はfloat型、ラベルはlong型（分類タスクの損失関数に必要）
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoaderを作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

**コラム：torch.Tensorだと何が嬉しいのか**
1. `X_train_tensor.to('cuda')` で GPUのサポートを受けられる
2. `requires_grad=True` で自動微分ができる

# 2. モデルの定義

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleMLP, self).__init__()
        # MLP層
        self.layer1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(64, num_classes) # num_classesは3 (3品種)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

In [ ]:
# モデルのインスタンス化
# 入力サイズは13 (ワインの特徴量数)、出力サイズは3 (等級数)
input_size = X_train.shape[1] # 13
num_classes = len(np.unique(y)) # 3
model = SimpleMLP(input_size, num_classes)

# 3. 学習の設定

In [ ]:
# 損失関数: CrossEntropyLoss (多クラス分類)
criterion = nn.CrossEntropyLoss()
# 最適化手法: Adam 一般的に高性能な最適化手法
optimizer = optim.Adam(model.parameters(), lr=0.01)
# 学習回数
num_epochs = 30

# 4. 学習ループ

In [ ]:
print("--- 学習開始 ---")
for epoch in range(num_epochs):
    model.train() # モデルを訓練モードに設定
    for inputs, labels in train_loader:
        # 勾配をゼロにリセット
        optimizer.zero_grad()
        
        # 順伝播
        outputs = model(inputs)
        
        # 損失の計算
        loss = criterion(outputs, labels)
        
        # 逆伝播
        loss.backward()

        # パラメータの更新
        optimizer.step()

    # 学習進捗の表示
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
print("--- 学習終了 ---")

In [ ]:
# 学習した重みパラメータの可視化 最初の層 (layer1)の例
state_dict = model.state_dict()
layer1_weight = state_dict['layer1.weight']
layer1_bias = state_dict['layer1.bias']

print("--- 2. 特定の層の重みとバイアスの内容 ---")

# 重み（一部表示）
print("\n[layer1.weight] (重み W) 最初の5行と5列だけ:")
print(layer1_weight[:5, :5]) 

# バイアス（全て表示）
print("\n[layer1.bias] (バイアス b):")
print(layer1_bias)

# 形状の確認
print(f"\nlayer1.weight の形状: {layer1_weight.shape}")
print(f"layer1.bias の形状: {layer1_bias.shape}")

# 5. 推論と評価

In [ ]:
# モデルを評価モードに設定（Dropoutなどの層を無効化）
model.eval()

# テストデータでの推論
with torch.no_grad(): # 勾配計算を無効化
    # 推論（テストデータ全体を一度に）
    test_outputs = model(X_test_tensor)
    
    # 予測クラスを取得
    # torch.max(test_outputs, 1) は (最大値, インデックス) を返す。インデックスが予測クラス。
    _, predicted = torch.max(test_outputs.data, 1)

In [ ]:
# PyTorchテンソルをNumPy配列に変換
y_pred = predicted.numpy()
y_true = y_test_tensor.numpy()

# 正解率の計算
accuracy = accuracy_score(y_true, y_pred)

print("\n--- 評価結果 ---")
print("施策: ベースライン")
print(f"テストデータの予測結果: {y_pred}")
print(f"テストデータの正解率 (Accuracy): {accuracy:.4f}")

### 練習問題
- みなさんは、AIエンジニアです。正解率90%のワインソムリエAIを作ってください。
- 精度が上がったら、評価結果をチャットに貼って報告してください。

**チャットのフォーマット**
```
--- 評価結果 ---
施策: XXXXの値をXXXXに変更
テストデータの予測結果: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
テストデータの正解率 (Accuracy): 0.3889
```

### ヒント
1. MLPの構造を変えてみましょう
    1. 層数
    1. 隠れ層の次元数
1. ハイパーパラメータを変えてみましょう
    1. 学習回数（エポック数）
    1. 学習率
    1. optimizerの種類
    1. 損失関数の種類